In [3]:
import requests
import os
import json
from api_keys import BEARER_TOKEN
from tqdm import tqdm
import os
import pandas as pd
from datetime import datetime



# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(query = "@BBB -is:retweet",until_id=None):
    
    #query = "@BBB -is:retweet"
    #"from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    
    if until_id:
    
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100&until_id={}".format(
            query,until_id
        )
        
    else:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100".format(
            query,until_id
        )
            
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def extract_100_tweets(query = "@BBB -is:retweet",until_id=None):
    bearer_token = BEARER_TOKEN
    url = create_url(query,until_id)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    data_tweets = json.dumps(json_response, indent=4, sort_keys=True)
    return json_response

def extract_many_tweets(qnt_cycle=10,folder="data_tweets"):
    
    
    oldest_id = None
    
    for i in tqdm(range(qnt_cycle)):
    
        
        if i == 0:
            
            #extract the 100 tweets first
        
            data_tweets = extract_100_tweets(query = "@BBB -is:retweet",until_id=None)
            
            df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
            
            #get the current date
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            
            oldest_id = data_tweets['meta']['oldest_id']
            
            oldest_date = date_extraction
            
            df_data_tweets = df_data_tweets_temp.copy()
            
            # name file
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,date_extraction_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file,sep=",")
            
    
            
        else:
            
            
            #extract more 100 tweets 
            
            data_tweets_temp = extract_100_tweets(query = "@BBB -is:retweet",until_id=oldest_id)
            
            df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
            
            
            #get the current date
            
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            oldest_id = data_tweets['meta']['oldest_id']
            
            df_data_tweets = pd.concat([df_data_tweets,df_data_tweets_temp.copy()])
            
            date_extraction = datetime.now()
            
            df_data_tweets.reset_index(inplace=True,drop=True)
            
            
            # remove old files
            
            os.remove(name_file)
            
            
            # name file
            
            oldest_date_str = str(oldest_date).replace(".","-").replace(":","-").replace(" ","-")
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,oldest_date_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file.format(folder),sep=",")
            
            

    return df_data_tweets
    
    


In [6]:
data_tweets_final = extract_many_tweets(qnt_cycle=1000,folder="data_tweets")

100%|██████████| 10/10 [00:10<00:00,  1.02s/it]


In [7]:
data_tweets_final

,id,text,date_extraction
0,1368679193194692613,@tweetaela @bbb Eu faria pior que a Juliete 🤣🤣...,2021-03-07 18:45:35.380495
1,1368679191546368003,@bbb Nossa campeã!!!@encantojuliette https://t...,2021-03-07 18:45:35.380495
2,1368679189159755777,@bbb Detalhe é que qndo a maior amiga dela vih...,2021-03-07 18:45:35.380495
3,1368679170688114689,@LaluManoban @beatrizrbbm @bbb Ridículos são v...,2021-03-07 18:45:35.380495
4,1368679150370820098,@bbb Perdeu minha admiração essa moça,2021-03-07 18:45:35.380495
...,...,...,...
995,1368678153380634631,@rpfgl @mosercoment @FCCaioeRodolfo1 @bbb Sara...,2021-03-07 18:45:44.489294
996,1368678127216562178,@bbb Prior fala sobre a saída de Karol Conká e...,2021-03-07 18:45:44.489294
997,1368678115946471425,@PyongLeeTV CAIO🌴 E RODOLFO🌴 NÃO GOSTA DO GIL ...,2021-03-07 18:45:44.489294
998,1368678109961158668,"@bbb SOFRIDAH KKKKKK... TE LIGA GAROTA, VC NAO...",2021-03-07 18:45:44.489294
